In [100]:
import math
import random

In [101]:
def euclidGcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = euclidGcd(b % a, a)
        return (g, x - (b // a) * y, y)

def moduloInverse(a, m):
    g, x, y = euclidGcd(a, m)
    if g != 1:
        raise Exception('modulo inverse does not exist')
    else:
        return x % m

In [102]:
#Function to check if a nummber is prime
def isPrime(n):
    if n == 2:
        return True
    
    if n%2 == 0:
        return False
    
    for i in range(3, math.ceil(math.sqrt(n))+1, 2):
        if n%i == 0:
            return False
        
    return True    

In [103]:
#Function to generate a random prime number of rho bits
def generatePrime(rho=16):
    
    temp = random.getrandbits(rho)
    temp = temp | (1<<(rho-1)) | 1
    
    while not isPrime(temp):
        temp = random.getrandbits(rho)
        temp = temp | (1<<(rho-1)) | 1
    
    return temp

In [104]:
p = generatePrime()
q = generatePrime()

while p == q:
    q = generatePrime()
    
N = p*q

phiN = (p-1)*(q-1)

In [105]:
numAttributes = 5
pi = [None] * numAttributes
qi = [None] * numAttributes

In [106]:
for i in range(0, numAttributes):
    temp = random.randint(2, N-1)
    
    while math.gcd(temp, phiN) != 1:
        temp = random.randint(2, N-1)

    pi[i] = temp  
    qi[i] = moduloInverse(p)

NameError: name 'modu' is not defined

In [107]:
pi

[2362992097, None, None, None, None]